# Common imports

In [ ]:
%matplotlib inline
import os
import sys
import glob
import re
import numpy as np
import random
import matplotlib
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image

# Provided functions
### Image loading and saving

In [ ]:
def LoadFaceImages(pathname, subject_name, num_images):
    """
    Load the set of face images.  
    The routine returns
        ambimage: image illuminated under the ambient lighting
        imarray: a 3-D array of images, h x w x Nimages
        lightdirs: Nimages x 3 array of light source directions
    """

    def load_image(fname):
        return np.asarray(Image.open(fname))

    def fname_to_ang(fname):
        yale_name = os.path.basename(fname)
        return int(yale_name[12:16]), int(yale_name[17:20])

    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z

    ambimage = load_image(
        os.path.join(pathname, subject_name + '_P00_Ambient.pgm'))
    im_list = glob.glob(os.path.join(pathname, subject_name + '_P00A*.pgm'))
#     print(len(im_list))
    if num_images <= len(im_list):
        im_sub_list = np.random.choice(im_list, num_images, replace=False)
    else:
        print(
            'Total available images is less than specified.\nProceeding with %d images.\n'
            % len(im_list))
        im_sub_list = im_list
    im_sub_list.sort()
    imarray = np.stack([load_image(fname) for fname in im_sub_list], axis=-1)
    Ang = np.array([fname_to_ang(fname) for fname in im_sub_list])

    x, y, z = sph2cart(Ang[:, 0] / 180.0 * np.pi, Ang[:, 1] / 180.0 * np.pi, 1)
    lightdirs = np.stack([y, z, x], axis=-1)
    return ambimage, imarray, lightdirs

In [ ]:
def save_outputs(subject_name, albedo_image, surface_normals):
    im = Image.fromarray((albedo_image*255).astype(np.uint8))
    im.save("%s_albedo.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,0]*128+128).astype(np.uint8))
    im.save("%s_normals_x.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,1]*128+128).astype(np.uint8))
    im.save("%s_normals_y.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,2]*128+128).astype(np.uint8))
    im.save("%s_normals_z.jpg" % subject_name)

### Plot the height map

In [ ]:
def set_aspect_equal_3d(ax):
    """https://stackoverflow.com/questions/13685386"""
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([
        abs(lim - mean_)
        for lims, mean_ in ((xlim, xmean), (ylim, ymean), (zlim, zmean))
        for lim in lims
    ])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])


def display_output(albedo_image, height_map):
    %matplotlib notebook
    fig = plt.figure()
    plt.imshow(albedo_image, cmap='gray')
    plt.axis('off')
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(20, 20)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

### Plot the surface norms. 

In [ ]:
def plot_surface_normals(surface_normals):
    """
    surface_normals: h x w x 3 matrix.
    """
    fig = plt.figure()
    ax = plt.subplot(1, 3, 1)
    ax.axis('off')
    ax.set_title('X')
    im = ax.imshow(surface_normals[:,:,0])
    ax = plt.subplot(1, 3, 2)
    ax.axis('off')
    ax.set_title('Y')
    im = ax.imshow(surface_normals[:,:,1])
    ax = plt.subplot(1, 3, 3)
    ax.axis('off')
    ax.set_title('Z')
    im = ax.imshow(surface_normals[:,:,2])

# Your implementation

In [ ]:
def preprocess(ambimage, imarray):
    """
    preprocess the data: 
        1. subtract ambient_image from each image in imarray.
        2. make sure no pixel is less than zero.
        3. rescale values in imarray to be between 0 and 1.
    Inputs:
        ambimage: h x w
        imarray: h x w x Nimages
    Outputs:
        processed_imarray: h x w x Nimages
    """
    

    temp = (imarray - ambimage[:,:,np.newaxis])/255

    processed_imarray = np.where(temp>0, temp, 0)

    return processed_imarray

In [ ]:
def photometric_stereo(imarray, light_dirs):
    """
    Inputs:
        imarray:  h x w x Nimages
        light_dirs: Nimages x 3
    Outputs:
        albedo_image: h x w
        surface_norms: h x w x 3
    """
    # Get image parameters
    h= imarray.shape[0]
    w= imarray.shape[1]
    N= imarray.shape[2]

    # Transform imarray into n x (h*w)
    imarray = np.transpose( imarray, (2, 0, 1))
    imarray = np.reshape(imarray,(N, h*w))
    
    # Compute g in 3 x (npixl)
    g= np.linalg.lstsq(light_dirs,imarray, rcond= None)[0]

    # Compute albedo

    temp_albedo = np.linalg.norm(g, axis = 0) # Compute maganitude
    albedo_image = np.reshape(temp_albedo,(h,w)) 
    
    # Compute surface_normals
    
    surface_normals = g/temp_albedo # (3 x pixel)
    surface_normals = np.transpose(surface_normals, (1,0)) # switch location
    surface_normals = np.reshape(surface_normals, (h,w,3))
    
    
    return albedo_image, surface_normals

In [ ]:
def get_surface(surface_normals, integration_method):
    """
    Inputs:
        surface_normals:h x w x 3
        integration_method: string in ['average', 'column', 'row', 'random']
    Outputs:
        height_map: h x w
    """
    h = surface_normals.shape[0]
    w = surface_normals.shape[1]
    
    f_x = surface_normals[:,:,0]/surface_normals[:,:,2] # (192 x 168)
    f_y = surface_normals[:,:,1]/surface_normals[:,:,2] # (192 x 168)
    
    if integration_method == 'row':
        temp = np.zeros((h,w))
        
        # Integrate the first row 
        temp[0, ::] = np.cumsum(f_x[0,::])
        temp[1::,:]= f_y[1::,:]
        # Integrate the column
        height_map = np.cumsum(temp, axis =0)
        
    elif integration_method == 'column':
        temp = np.zeros((h,w))
        
        # Integrate the first column
        temp[:, 0] = np.cumsum(f_y[:,0])
        temp[:,1::] = f_x[:,1::]
        # Integrate rows
        height_map = np.cumsum(temp, axis = 1)
    
    elif integration_method == 'average':
        t1= np.zeros((h,w))
        t2= np.zeros((h,w))
        
        
        t1[0, ::] = np.cumsum(f_x[0,::])
        t1[1::,:] = f_y[1::,:]
        map_1 = np.cumsum(t1,axis = 0)
        
        t2[::, 0] = np.cumsum(f_y[::,0])
        t2[:,1::] = f_x[:,1::]
        map_2 = np.cumsum(t2, axis = 1)
        
        # Take average
        height_map = (map_1 + map_2)/2

    elif integration_method == 'random':
        n = 20
        height_map = np.zeros((h,w))
        for row in range(h):
            for col in range(w):
                # Top left most(0,0), starting point
                if (row == 0 and col == 0):
                    height_map[row, col] = 0
                else:
                    for i in range(n):
                        temp_sum = 0
                        step = 0
                        
                        direction = np.random.randint(0, 2 , size=col+row)
                        
                        # Starting point
                        x = 0
                        y = 0
                        
                        while(x < col or y < row):
                            # 0 horizontal, 1 vertical, x horizontal, y vertical
                            if x >= col:
                                direction[step]=1
                            if y >= row:
                                direction[step] = 0
                                
                            if direction[step] == 0:
                                temp_sum += f_x[y,x]
                                x = x + 1;
                            else:
                                temp_sum += f_y[y,x]
                                y = y + 1
                            step = step + 1;
                        height_map[row, col] = temp_sum + height_map[row, col]
                    height_map[row, col] = height_map[row, col] / n
    return height_map

# Main function

In [ ]:
root_path = 'croppedyale/croppedyale/'
subject_name_1 = 'yaleB05/'
subject_name = 'yaleB05'
integration_method = 'average'
save_flag = True

full_path = '%s%s' % (root_path, subject_name_1)

ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,
                                                    64)

processed_imarray = preprocess(ambient_image, imarray)

albedo_image, surface_normals = photometric_stereo(processed_imarray,
                                                   light_dirs)

start_time= time.clock()
height_map = get_surface(surface_normals, 'random')
print("--- %s seconds ---" % (time.clock() - start_time))

if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [ ]:
plot_surface_normals(surface_normals)

In [ ]:
display_output(albedo_image, height_map)